# Guide Complet : Calcul de l'Influence avec Influenciae

Ce notebook explique en détail comment les fonctions d'influence sont calculées dans Influenciae, en partant de la définition mathématique jusqu'à l'implémentation.

**Ce que vous allez apprendre :**

1. **D'où vient la formule d'influence** - Dérivation mathématique complète
2. **Pourquoi on utilise Fisher/Gauss-Newton** au lieu de la Hessienne exacte
3. **Comment KFAC factorise** la matrice de Fisher par couche
4. **Comment EKFAC corrige** les valeurs propres de KFAC
5. **Comment Influenciae implémente** ces concepts en TensorFlow

---

## Table des Matières

1. [Qu'est-ce que l'influence ?](#1.-Qu'est-ce-que-l'influence-?)
2. [Dérivation mathématique de la formule d'influence](#2.-Dérivation-mathématique)
3. [Pourquoi Fisher au lieu de la Hessienne ?](#3.-Pourquoi-Fisher-?)
4. [KFAC : Factorisation de Kronecker par couche](#4.-KFAC)
5. [EKFAC : Correction des valeurs propres](#5.-EKFAC)
6. [Implémentation dans Influenciae](#6.-Implémentation)
7. [Exemple pratique complet](#7.-Exemple-pratique)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, Reduction

# Imports influenciae
from deel.influenciae.common import InfluenceModel, FisherIHVP, KFACIHVP, EKFACIHVP
from deel.influenciae.common.inverse_hessian_vector_product import ExactIHVP

print(f"TensorFlow version: {tf.__version__}")

---

## 1. Qu'est-ce que l'influence ? <a name="1.-Qu'est-ce-que-l'influence-?"></a>

### Le problème : comprendre l'impact des données d'entraînement

L'**influence** cherche à approximer le **Leave-one-out**, c'est-à-dire à estimer l'**impact qu'aurait un exemple d'entraînement sur la perte d'un exemple de test**.

Les fonctions d'influence permettent de répondre à :
- Quels exemples d'entraînement ont été **utiles** à une prédiction ?
- Le modèle s'est trompé : sur quels exemples s'est-il **basé** pour cette mauvaise prédiction ?
- Est-ce que je devrais **ajouter** cet exemple pour améliorer une prédiction ?

### Définition formelle

L'influence de $z_{\rm train}$ sur $z_{\rm test}$ se définit comme :

$$\mathrm{Influence}(z_{\rm train}\to z_{\rm test}) = \left.\frac{d}{d\varepsilon}\,\mathcal{L}\bigl(z_{\rm test},\,\theta_\varepsilon(z_{\text{train}})\bigr)\right|_{\varepsilon=0}$$

**Interprétation** : Elle mesure la sensibilité de la loss de $z_{\rm test}$ à un **"up-weight" infinitésimal** de la loss de $z_{\rm train}$.

### Notations

| Symbole | Signification |
|---------|---------------|
| $\hat{\theta}$ | Poids optimaux du modèle pré-entraîné |
| $\theta_\varepsilon(z_{\text{train}})$ | Poids "modifiés" après up-weight de $\varepsilon$ sur $z_{\text{train}}$ |
| $\mathcal{L}(z, \theta)$ | Loss du modèle sur l'exemple $z$ avec les poids $\theta$ |
| $H_\theta$ | Matrice Hessienne (dérivée seconde par rapport à $\theta$) |

---

## 2. Dérivation mathématique de la formule d'influence <a name="2.-Dérivation-mathématique"></a>

On veut calculer :
$$\frac{d}{d\varepsilon}\, \mathcal{L}\bigl(z_{\rm test},\ \theta_\varepsilon(z_{\text{train}})\bigr) \Big|_{\varepsilon=0}$$

### 2.1 Décomposition avec la chain rule

On peut voir cette expression comme $\frac{d}{d\varepsilon} f(g(\varepsilon))$ avec :
- $g(\varepsilon) = \theta_\varepsilon(z_{\text{train}})$ (les poids après up-weight)
- $f(\theta) = \mathcal{L}(z_{\rm test}, \theta)$ (la loss de test)

Par la **chain rule** :

$$\boxed{\frac{d}{d\varepsilon}\, \mathcal{L}\bigl(z_{\rm test},\ \theta_\varepsilon\bigr) \Big|_{\varepsilon=0} = \nabla_\theta \mathcal{L}(z_{\rm test}, \hat{\theta}) \times \frac{d\theta_\varepsilon}{d\varepsilon}\Big|_{\varepsilon=0}}$$

Il nous faut donc calculer $\frac{d\theta_\varepsilon}{d\varepsilon}\Big|_{\varepsilon=0}$.

### 2.2 Qu'est-ce que $\theta_\varepsilon(z_{\text{train}})$ ?

Pour faire l'"up-weight" de $z_{\text{train}}$ de $\varepsilon$, on perturbe la loss globale :

$$R_\varepsilon(\theta, z_{\text{train}}) = \frac{1}{n}\sum_{i=1}^n \mathcal{L}(z_i,\theta) + \varepsilon\,\mathcal{L}(z_{\rm train},\theta)$$

Puis on cherche les poids $\theta_\varepsilon$ qui **minimisent cette nouvelle loss** :

$$\theta_\varepsilon(z_{\text{train}}) = \arg\min_{\theta}\;R_\varepsilon(\theta, z_{\text{train}})$$

Ce qui revient à résoudre (condition d'optimalité du premier ordre) :

$$\nabla_\theta R_\varepsilon\bigl(\theta_\varepsilon\bigr) = 0$$

### 2.3 Approximation de Taylor à l'ordre 1

Pour un petit $\varepsilon$ (proche de 0), $\theta_\varepsilon$ est proche de $\hat{\theta}$. On développe avec **Taylor** :

$$\nabla_\theta R_\varepsilon\bigl(\theta_\varepsilon\bigr) \approx \nabla_\theta R_\varepsilon(\hat{\theta}) + \nabla^2_\theta R_\varepsilon(\hat{\theta}) \cdot (\theta_\varepsilon - \hat{\theta})$$

En utilisant $\nabla_\theta R_\varepsilon(\theta_\varepsilon) = 0$ et en isolant $(\theta_\varepsilon - \hat{\theta})$ :

$$\theta_\varepsilon - \hat{\theta} \approx - \left[ \nabla^2_\theta R_\varepsilon(\hat{\theta}) \right]^{-1} \nabla_\theta R_\varepsilon(\hat{\theta})$$

Or, puisque $\hat{\theta}$ minimise la loss originale, on a $\frac{1}{n}\sum_{i=1}^n \nabla_\theta \mathcal{L}(z_i,\hat{\theta}) = 0$.

Donc :
$$\nabla_\theta R_\varepsilon(\hat{\theta}) = \varepsilon\,\nabla_\theta \mathcal{L}(z_{\rm train},\hat{\theta})$$

Et à l'ordre 1 en $\varepsilon$ :
$$\nabla^2_\theta R_\varepsilon(\hat{\theta}) \approx \underbrace{\frac{1}{n}\sum_{i=1}^n \nabla^2_\theta \mathcal{L}(z_i,\hat{\theta})}_{= H_\theta(\hat{\theta})}$$

### 2.4 Résultat : formule de $\frac{d\theta_\varepsilon}{d\varepsilon}$

En combinant et en dérivant par rapport à $\varepsilon$ :

$$\boxed{\frac{d\theta_\varepsilon}{d\varepsilon}\Big|_{\varepsilon=0} = - H_\theta^{-1}(\hat{\theta}) \cdot \nabla_\theta \mathcal{L}(z_{\rm train},\hat{\theta})}$$

### 2.5 Formule finale de l'influence

En multipliant par $\nabla_\theta \mathcal{L}(z_{\rm test}, \hat{\theta})$ (chain rule de l'étape 2.1) :

$$\boxed{\mathrm{Influence}(z_{\rm train} \to z_{\rm test}) = -\nabla_\theta \mathcal{L}(z_{\rm test}, \hat{\theta})^T \cdot H_\theta^{-1}(\hat{\theta}) \cdot \nabla_\theta \mathcal{L}(z_{\rm train}, \hat{\theta})}$$

### Structure du calcul

```
1. grad_test = nabla_theta L(z_test)        # Gradient sur le test
2. ihvp = H^{-1} @ grad_test                # IHVP (Inverse Hessian Vector Product)
3. Pour chaque z_train :
   a. grad_train = nabla_theta L(z_train)   # Gradient sur le train
   b. influence = -ihvp^T @ grad_train      # Produit scalaire
```

**Le défi principal** : calculer $H^{-1} \cdot v$ efficacement (IHVP) !

---

## 3. Pourquoi Fisher au lieu de la Hessienne ? <a name="3.-Pourquoi-Fisher-?"></a>

### 3.1 Problème avec la Hessienne exacte

La Hessienne exacte nécessite une **double backpropagation** très coûteuse :

$$H_\theta = \nabla^2_\theta \mathcal{L} = \underbrace{J_{y\theta}^T \, H_y \, J_{y\theta}}_{\text{Gauss-Newton (facile)}} + \underbrace{\sum_i \frac{\partial \mathcal{L}}{\partial y_i} \nabla^2_\theta y_i}_{\text{Cauchemar computationnel}}$$

Où :
- $J_{y\theta} = \frac{\partial y}{\partial \theta}$ : Jacobienne des sorties du modèle
- $H_y = \frac{\partial^2 \mathcal{L}}{\partial y^2}$ : Hessienne de la loss par rapport aux sorties
- Le dernier terme nécessite les dérivées secondes à travers tout le réseau !

### 3.2 Approximation Gauss-Newton

On ignore le terme coûteux et on garde :

$$H_{\theta} \approx G_\theta = J_{y\theta}^T \, H_y \, J_{y\theta}$$

C'est la **Gauss-Newton Hessian (GNH)**, aussi appelée **matrice de Fisher empirique**.

### 3.3 Lien avec les gradients

Par la chain rule, le gradient de la loss est :
$$g = \nabla_\theta \mathcal{L} = J_{y\theta}^T \cdot \nabla_y \mathcal{L}$$

Donc :
$$g \, g^T = J_{y\theta}^T \, (\nabla_y \mathcal{L})(\nabla_y \mathcal{L})^T \, J_{y\theta}$$

Pour des losses de type cross-entropy, on a $\mathbb{E}[(\nabla_y \mathcal{L})(\nabla_y \mathcal{L})^T] \approx H_y$.

D'où **l'approximation Fisher** :

$$\boxed{H_\theta \approx F = \mathbb{E}[g \, g^T] = \frac{1}{n}\sum_{i=1}^n \nabla_\theta \mathcal{L}(z_i) \cdot \nabla_\theta \mathcal{L}(z_i)^T}$$

### 3.4 Avantages de Fisher

| Aspect | Hessienne exacte | Fisher |
|--------|------------------|--------|
| **Calcul** | Double backprop | Produit externe de gradients |
| **Définie positive** | Pas toujours | Toujours ! |
| **Inversibilité** | Problématique | Avec damping, OK |

### 3.5 Damping

Comme les réseaux ne sont pas convexes, on ajoute un terme de **damping** $\lambda$ :

$$H^{-1} \approx (G + \lambda I)^{-1}$$

Cela garantit l'inversibilité et stabilise le calcul.

---

## 4. KFAC : Factorisation de Kronecker par couche <a name="4.-KFAC"></a>

### 4.1 Le problème de mémoire

Même la matrice de Fisher est énorme :
- Pour $p$ paramètres : $F$ a taille $p \times p$
- Un réseau de 1M paramètres : $F$ = 4 TB de mémoire !

### 4.2 Idée clé : structure bloc-diagonale

KFAC exploite la structure par couches :

$$G \approx \begin{bmatrix} G_{1,1} & 0 & \cdots \\ 0 & G_{2,2} & \\ \vdots & & \ddots \end{bmatrix}$$

On ignore les interactions **entre** couches et on ne garde que les blocs **diagonaux** $G_{l,l}$ pour chaque couche $l$.

### 4.3 Factorisation de Kronecker pour une couche Dense

Pour une couche Dense $h = Wa + b$ :
- $a$ : activation d'**entrée** (dimension $d_{in}$)
- $h$ : pré-activation de **sortie** (dimension $d_{out}$)
- $\delta = \frac{\partial \mathcal{L}}{\partial h}$ : gradient par rapport à la sortie

Le gradient par rapport aux poids est un **produit externe** :
$$\nabla_W \mathcal{L} = a \cdot \delta^T$$

En vectorisant : $\text{vec}(\nabla_W \mathcal{L}) = \delta \otimes a$

**Approximation KFAC** :
$$F_W \approx \underbrace{\mathbb{E}[\delta \delta^T]}_{G \;(d_{out} \times d_{out})} \;\otimes\; \underbrace{\mathbb{E}[aa^T]}_{A \;(d_{in} \times d_{in})}$$

### 4.4 Réduction de mémoire

Au lieu de stocker $F$ de taille $(d_{in} \cdot d_{out})^2$, on stocke :
- $A$ : $d_{in}^2$ éléments
- $G$ : $d_{out}^2$ éléments

**Exemple** : couche 1000 $\to$ 1000
- Fisher complète : $10^{12}$ éléments
- KFAC : $2 \times 10^6$ éléments (**500 000x** moins !)

### 4.5 Inversion avec le produit de Kronecker

Grâce à la propriété : $(G \otimes A)^{-1} = G^{-1} \otimes A^{-1}$

On inverse deux **petites** matrices au lieu d'une grande !

### 4.6 Pi-damping

Le damping standard $(G \otimes A + \lambda I)$ n'est pas factorisable.

KFAC utilise le **$\pi$-damping** :

$$\pi = \sqrt{\frac{\text{tr}(A) / \dim(A)}{\text{tr}(G) / \dim(G)}}$$

$$G \otimes A + \lambda I \approx \left(G + \frac{\sqrt{\lambda}}{\pi} I\right) \otimes \left(A + \sqrt{\lambda} \cdot \pi \cdot I\right)$$

Le rôle de $\pi$ est d'**équilibrer** le damping entre $A$ et $G$ selon leurs échelles spectrales.

### 4.7 Résolution du système KFAC

Pour résoudre $(G \otimes A + \lambda I) \cdot x = v$ :

1. Reshape $v$ en matrice $V$ de taille $(d_{in}, d_{out})$
2. $A_{reg} = A + \sqrt{\lambda} \cdot \pi \cdot I$, $G_{reg} = G + \frac{\sqrt{\lambda}}{\pi} \cdot I$
3. Résoudre $G_{reg} \cdot X = V^T$ pour $X$
4. Résoudre $A_{reg} \cdot Y = X^T$ pour $Y$
5. La solution est $\text{vec}(Y)$

In [ ]:
# Démonstration du produit de Kronecker
def demo_kronecker_product():
    """
    Illustre la factorisation de Kronecker utilisée par KFAC.
    """
    # Petites matrices pour démonstration
    A = np.array([[1, 2], [3, 4]], dtype=np.float32)
    G = np.array([[5, 6], [7, 8]], dtype=np.float32)
    
    # Produit de Kronecker
    K = np.kron(G, A)
    print("Produit de Kronecker: G x A")
    print(f"A shape: {A.shape}, G shape: {G.shape}")
    print(f"G x A shape: {K.shape}")
    print()
    
    # Vérification: (G x A)^{-1} = G^{-1} x A^{-1}
    K_inv_direct = np.linalg.inv(K)
    K_inv_kron = np.kron(np.linalg.inv(G), np.linalg.inv(A))
    
    print("Verification: (G x A)^{-1} = G^{-1} x A^{-1}")
    print(f"Erreur max: {np.max(np.abs(K_inv_direct - K_inv_kron)):.2e}")
    
demo_kronecker_product()

---

## 5. EKFAC : Correction des valeurs propres <a name="5.-EKFAC"></a>

### 5.1 Limitation de KFAC

KFAC suppose que les valeurs propres de $G \otimes A$ sont les produits des valeurs propres individuelles :

$$\lambda_{ij}^{KFAC} = \lambda_i^G \cdot \lambda_j^A$$

C'est une **approximation** qui peut être inexacte.

### 5.2 Principe d'EKFAC

EKFAC (Eigenvalue-corrected KFAC) calcule les **vraies valeurs propres** dans la base propre de Kronecker.

**Étape 1 : Décomposition spectrale**

$$A = U_A \Lambda_A U_A^T, \quad G = U_G \Lambda_G U_G^T$$

**Étape 2 : Projection dans la base propre**

Pour chaque échantillon :
- $a_{kfe} = U_A^T \cdot a$ (activation projetée)
- $\delta_{kfe} = U_G^T \cdot \delta$ (gradient projeté)

**Étape 3 : Calcul des vraies valeurs propres**

$$\Lambda_{corr}[i,j] = \frac{1}{n} \sum_{samples} (\delta_{kfe}[i])^2 \cdot (a_{kfe}[j])^2$$

### 5.3 Résolution EKFAC

Dans la base propre, le système est **diagonal** :

$$(\Lambda_{corr} + \lambda I) \cdot x_{kfe} = v_{kfe}$$

Solution élémentaire :
$$x_{kfe}[i,j] = \frac{v_{kfe}[i,j]}{\Lambda_{corr}[i,j] + \lambda}$$

Puis transformation inverse : $x = (U_A \otimes U_G) \cdot x_{kfe}$

### 5.4 Comparaison KFAC vs EKFAC

| Aspect | KFAC | EKFAC |
|--------|------|-------|
| **Valeurs propres** | $\lambda^G_i \cdot \lambda^A_j$ (approximées) | Empiriques (exactes) |
| **Coût** | 1 passe sur les données | 2 passes (facteurs + eigenvalues) |
| **Précision** | Bonne | Meilleure |

---

## 6. Implémentation dans Influenciae <a name="6.-Implémentation"></a>

### 6.1 La classe `InfluenceModel`

Influenciae utilise un wrapper `InfluenceModel` pour sélectionner les poids à surveiller :

In [ ]:
# Creation d'un modele exemple
def create_example_model():
    """Cree un MLP simple pour demonstration."""
    inputs = layers.Input(shape=(100,), name="input")
    
    # Couche "feature extractor" (on pourrait vouloir l'ignorer)
    x = layers.Dense(64, activation="relu", name="feature_extractor")(inputs)
    
    # Couches de classification (on veut surveiller celles-ci)
    x = layers.Dense(32, activation="relu", name="hidden")(x)
    outputs = layers.Dense(10, activation=None, name="classifier")(x)
    
    return Model(inputs=inputs, outputs=outputs, name="example_mlp")

model = create_example_model()
model.summary()

In [ ]:
# Loss function SANS reduction (obligatoire pour influenciae)
loss_fn = SparseCategoricalCrossentropy(from_logits=True, reduction=Reduction.NONE)

def process_batch_for_loss(batch):
    """Convertit un batch en format (inputs, labels, sample_weights)."""
    inputs, labels = batch
    sample_weights = tf.ones_like(labels, dtype=tf.float32)
    return inputs, labels, sample_weights

# Surveiller SEULEMENT les couches de classification (hidden + classifier)
influence_model = InfluenceModel(
    model,
    start_layer="hidden",    # Par nom de couche
    last_layer="classifier",
    loss_function=loss_fn,
    process_batch_for_loss_fn=process_batch_for_loss
)

print(f"Nombre de parametres surveilles: {influence_model.nb_params}")
print(f"Poids surveilles: {[w.name for w in influence_model.weights]}")

### 6.2 `tf.GradientTape(persistent=True)` : l'équivalent des hooks PyTorch

Pour KFAC/EKFAC, on a besoin de :
1. **Activations d'entrée** $a$ de chaque couche
2. **Gradients de sortie** $\delta$ de chaque couche

En PyTorch, on utilise des **hooks**. En TensorFlow, on utilise `GradientTape(persistent=True)` :

In [ ]:
# Demonstration de la capture des gradients
x_sample = tf.random.normal((1, 100))
y_sample = tf.constant([3])

classifier_layer = model.get_layer("classifier")

# 1. Creer un sous-modele pour obtenir l'activation d'entree de la couche
layer_input_model = tf.keras.Model(
    inputs=model.input,
    outputs=classifier_layer.input
)

# 2. Utiliser persistent=True pour pouvoir calculer plusieurs gradients
with tf.GradientTape(persistent=True) as tape:
    output = model(x_sample)
    loss = loss_fn(y_sample, output)
    loss_scalar = tf.reduce_sum(loss)

# 3. Recuperer a (activation d'entree de la couche)
a = layer_input_model(x_sample)  # Shape: (1, in_features)
a = tf.squeeze(a, axis=0)  # Shape: (in_features,)

# 4. Recuperer delta = dL/dh via le gradient du bias
#    Pour Dense: y = Wa + b, donc dL/db = dL/dy = delta
delta = tape.gradient(loss_scalar, classifier_layer.bias)

# 5. On peut aussi obtenir le gradient du kernel
grad_kernel = tape.gradient(loss_scalar, classifier_layer.kernel)

del tape

print(f"Activation d'entree (a): shape={a.shape}")
print(f"Gradient de sortie (delta = dL/dy): shape={delta.shape}")
print(f"Gradient du kernel (dL/dW): shape={grad_kernel.shape}")
print(f"\nVerification: dL/dW = a x delta (produit externe)")
print(f"  grad_W ~= outer(a, delta): {np.allclose(grad_kernel.numpy(), np.outer(a, delta), rtol=0.1)}")

### 6.3 Les calculateurs IHVP dans Influenciae

| Classe | Méthode | Complexité mémoire |
|--------|---------|-------------------|
| `ExactIHVP` | Hessienne exacte | $O(p^2)$ |
| `FisherIHVP` | $F = \mathbb{E}[gg^T]$ | $O(p^2)$ |
| `KFACIHVP` | $F \approx G \otimes A$ | $O(\sum d_{in}^2 + d_{out}^2)$ |
| `EKFACIHVP` | KFAC + eigenvalues corrigées | $O(\sum d_{in}^2 + d_{out}^2)$ |

In [ ]:
# Creer des donnees pour la demonstration
np.random.seed(42)
n_samples = 50
X_train = np.random.randn(n_samples, 100).astype(np.float32)
y_train = np.random.randint(0, 10, n_samples).astype(np.int32)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)

# Creer EKFACIHVP
print("Creation de EKFACIHVP...")
ekfac_ihvp = EKFACIHVP(
    model=influence_model,
    train_dataset=train_dataset,
    damping=1e-3,
    update_eigen=True  # Calculer les vraies valeurs propres
)

print(f"\nBlocs KFAC calcules!")
for layer_idx, (A, G) in ekfac_ihvp.kfac_blocks.items():
    layer_info = ekfac_ihvp.layer_info[layer_idx]
    print(f"\nLayer {layer_idx}: {layer_info['layer'].name}")
    print(f"  A (covariance activation): shape={A.shape}")
    print(f"  G (covariance gradient):   shape={G.shape}")
    print(f"  Eigenvalues (EKFAC):       shape={ekfac_ihvp.evals[layer_idx].shape}")

---

## 7. Exemple pratique complet <a name="7.-Exemple-pratique"></a>

Mettons tout ensemble pour calculer l'influence :

In [ ]:
from deel.influenciae.influence import FirstOrderInfluenceCalculator
from deel.influenciae.utils import ORDER

# 1. Preparer les donnees
np.random.seed(42)
n_train, n_test = 100, 10

X_train = np.random.randn(n_train, 100).astype(np.float32)
y_train = np.random.randint(0, 10, n_train).astype(np.int32)

X_test = np.random.randn(n_test, 100).astype(np.float32)
y_test = np.random.randint(0, 10, n_test).astype(np.int32)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(10)
hessian_dataset = tf.data.Dataset.from_tensor_slices((X_train[:50], y_train[:50])).batch(8)

print(f"Train: {n_train} samples, Test: {n_test} samples")

In [ ]:
# 2. Creer et entrainer un modele
model = create_example_model()
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.fit(X_train, y_train, epochs=5, verbose=0)
print(f"Accuracy: {model.evaluate(X_test, y_test, verbose=0)[1]:.2%}")

In [ ]:
# 3. Wrapper le modele avec InfluenceModel
influence_model = InfluenceModel(
    model,
    start_layer="hidden",
    last_layer="classifier",
    loss_function=loss_fn,
    process_batch_for_loss_fn=process_batch_for_loss
)
print(f"Parametres surveilles: {influence_model.nb_params}")

In [ ]:
# 4. Creer le calculateur IHVP (EKFAC)
ihvp_calculator = EKFACIHVP(
    model=influence_model,
    train_dataset=hessian_dataset,
    damping=1e-3,
    update_eigen=True
)
print("EKFAC IHVP calculator cree!")

In [ ]:
# 5. Creer le calculateur d'influence
influence_calculator = FirstOrderInfluenceCalculator(
    influence_model,
    train_dataset,
    ihvp_calculator
)
print("FirstOrderInfluenceCalculator cree!")

In [ ]:
# 6. Calculer les exemples les plus influents
top_k = 5

print(f"\nRecherche des top-{top_k} exemples influents...")
top_k_results = influence_calculator.top_k(
    test_dataset,
    train_dataset,
    k=top_k,
    order=ORDER.DESCENDING
)

# Afficher les resultats
for batch_result in top_k_results:
    test_samples, influence_values, train_samples = batch_result
    test_inputs, test_labels = test_samples
    
    for i in range(min(3, len(test_labels))):
        print(f"\nExemple de test {i} (label={test_labels[i].numpy()}):")
        print(f"  Top-{top_k} influences: {influence_values[i].numpy()}")

---

## Resume : Pipeline complet de calcul de l'influence

```
1. ENTRAINER le modele
         |
         v
2. CALCULER les facteurs KFAC/EKFAC sur le dataset d'entrainement
   - Pour chaque couche Dense : extraire A et G
   - EKFAC : calculer les valeurs propres corrigees
         |
         v
3. Pour chaque echantillon de TEST z_test :
   a) Calculer le gradient : v = nabla_theta L(z_test)
   b) Calculer l'IHVP : ihvp = H^{-1} @ v  (via KFAC/EKFAC)
         |
         v
4. Pour chaque echantillon d'ENTRAINEMENT z_train :
   a) Calculer le gradient : g = nabla_theta L(z_train)
   b) Calculer l'influence : I(z_train, z_test) = -ihvp^T @ g
```

### Formules cles

| Concept | Formule |
|---------|--------|
| **Influence** | $I(z_{train} \to z_{test}) = -\nabla L_{test}^T \cdot H^{-1} \cdot \nabla L_{train}$ |
| **Fisher** | $F = \mathbb{E}[g \cdot g^T]$ |
| **KFAC** | $F \approx G \otimes A$ avec $A = \mathbb{E}[aa^T]$, $G = \mathbb{E}[\delta\delta^T]$ |
| **EKFAC** | Comme KFAC mais avec $\Lambda_{corr}[i,j] = \mathbb{E}[(\delta_{kfe}[i])^2 (a_{kfe}[j])^2]$ |

---

## References

- [Koh & Liang (2017)](https://arxiv.org/abs/1703.04730) - Understanding Black-box Predictions via Influence Functions
- [Martens & Grosse (2015)](https://arxiv.org/abs/1503.05671) - Optimizing Neural Networks with Kronecker-factored Approximate Curvature (KFAC)
- [George et al. (2018)](https://arxiv.org/abs/1806.03884) - Fast Approximate Natural Gradient Descent in a Kronecker-factored Eigenbasis (EKFAC)
- [Grosse et al. (2023)](https://arxiv.org/abs/2308.03296) - Studying Large Language Model Generalization with Influence Functions